In [9]:
# import required libraries

# for handling data retrieval from Binance Exchange
import binance
from binance.client import Client

import requests

url = 'https://api.binance.com/api/v3/klines'
params = {
  'symbol': 'BTCUSDT',
  'interval': '1h',
  'startTime': 1631041200000,
  'endTime':1631044799999

}
response = requests.get(url, params=params)
print(response.json())

[[1631041200000, '46922.02000000', '47240.04000000', '46515.68000000', '46694.59000000', '2513.80486000', 1631044799999, '117936708.87113800', 83161, '1220.24093000', '57254878.72015970', '0']]


In [ ]:
# This is for checking the library location
# print(binance.__file__)